In [10]:
import pandas as pd 
demand_df = pd.read_csv("demand3.csv")
supply_df = pd.read_csv("supply3.csv")


In [11]:
import itertools 
import requests
import json

import psycopg2
import pandas as pd
from random import randint
import requests
import json

conn_string = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

def cartesian(df1, df2):
    rows = itertools.product(df1.iterrows(), df2.iterrows())
    df = pd.DataFrame(left.append(right) for (_, left), (_, right) in rows)
    return df.reset_index(drop=True)

def get_distance_duration_from_google_maps(source_lat, source_lng, dest_lat, dest_lng):
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins={source_lat},{source_lng}&destinations={dest_lat},{dest_lng}"\
    .format(**locals())
    
    r = requests.get(url)
    d = json.loads(r.content)
    
    return_dict = {}
    
    return_dict["distance_route_km"] = d["rows"][0]["elements"][0]["distance"]["value"]/1000.0
    return_dict["duration_min"] = d["rows"][0]["elements"][0]["duration"]["value"]/60.

    return return_dict

def get_distance_crow_flies(source_lat, source_lng, dest_lat, dest_lng):

    sql = """
        select st_distance(
            ST_Transform(ST_SetSRID(ST_MakePoint({source_lng}, {source_lat}), 4326), 27700),
            ST_Transform(ST_SetSRID(ST_MakePoint({dest_lng}, {dest_lat}), 4326), 27700)
        ) as distance
    """
    
    cur.execute(sql.format(**locals()))
    crow =  cur.fetchone()[0]/1000.0
    return {"distance_crowflies_km":  crow, "duration_min":  crow, "distance_route_km":  crow}

In [12]:
all_data = cartesian(demand_df, supply_df)

In [13]:
def lat_lng_to_metrics(source_lat, source_lng, dest_lat, dest_lng):
    """
    Given source and dest coordinates, get travel time, route distance, 
    and distance as the crow flies and return as dict
    """
#     d1 = get_distance_duration_from_google_maps(source_lat, source_lng, dest_lat, dest_lng)
    d1 = get_distance_crow_flies(source_lat, source_lng, dest_lat, dest_lng)
#     d1.update(d2)
    return d1   

In [14]:
for r in all_data.iterrows():
    index = r[0]
    row = r[1]
    
    metrics = lat_lng_to_metrics(row["demand_lat"], row["demand_lng"], row["supply_lat"], row["supply_lng"])
    
    for key, value in metrics.iteritems(): 
        all_data.loc[index, key] = value

In [15]:
all_data.to_csv("all_data3.csv", index=False)